### COVID-19 Pandemic - Disease comparison, Resource Utilization and Financial Impact - A perspective

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
# Import a bunch of libraries.
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
from plotnine import *
import plotly.express as px
import plotly.io as pio
import folium
import requests

# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# storing and anaysis
import numpy as np
import pandas as pd

# visualization
import altair as alt

# display multiple outputs from a single cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# support output to HTML
from IPython.display import Javascript
from IPython.core.display import display, HTML

companies = ['FB','GOOGL','MSFT','JPM','BAC','WFC','JNJ','CVS','MRK','UAL','DAL','AAL','MAR','HLT','H']

In [ ]:
#VULNERABLE - COUNTIES - PRESENTATION - START
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import altair as alt # For displaying graphs
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import panel as pn
import datetime as dt
from altair import datum
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import requests
for dirname, _, filenames in os.walk('./icu-data/'):
    for filename in filenames:
        file = os.path.join(dirname, filename)
        #print(file)

# Any results you write to the current directory are saved as output.


##Financials Notebook - Start
listofdf = []
days = -70

for item in companies:
    histprices = requests.get(f"https://financialmodelingprep.com/api/v3/historical-price-full/{item}?serietype=line")
    histprices = histprices.json()

    #Parse the API response and select only last n days of prices
    histprices = histprices['historical'][days:]
    histpricesdf = pd.DataFrame.from_dict(histprices)
    histpricesdf['symbol']= item
    histpricesdf = histpricesdf.rename({'close': 'price'}, axis=1) 
    
    #append all dfs to list
    listofdf.append(histpricesdf)


In [ ]:
indexes = ['^DJI','^GSPC','^IXIC']

#,'JPM','BAC','JNJ','CVS','UA','DAL','MAR','HLT',,'^DJI','^GSPC','^IXIC','^VIX','^N100','^GDAXI','^N225','^HSI'

listofdf_index = []
days = 70

for item in indexes:
    histprices = requests.get(f"https://financialmodelingprep.com/api/v3/historical-price-full/index/{item}?serietype=line")
    histprices = histprices.json()

    #Parse the API response and select only last n days of prices
    histprices = histprices['historical'][:days]
    histpricesdf = pd.DataFrame.from_dict(histprices)
    histpricesdf['symbol']= item
    histpricesdf = histpricesdf.rename({'close': 'price'}, axis=1) 
    
    #append all dfs to list
    listofdf_index.append(histpricesdf)    

In [ ]:
dfs = [df.set_index('date') for df in listofdf]
histpriceconcat1 = pd.concat(listofdf,axis=0, sort=True)

#histpriceconcat1

In [ ]:

dfs = [df.set_index('date') for df in listofdf_index]
index= pd.concat(listofdf_index,axis=0, sort=True)
#index

In [ ]:
df_all=[histpriceconcat1,index]
df_all2=pd.concat(df_all,axis=0, sort=True)
#df_all2

In [ ]:
first_price=df_all2.loc[df_all2['date'] == '2020-01-07']
first_price2=first_price.rename({'price': 'first_price'}, axis=1).drop(['date'], axis=1)
#first_price2

In [ ]:
result = pd.merge(df_all2, first_price2, on=['symbol'])
result['change'] = result['price']/result['first_price']-1
result2=result.round(2)
####result2=result.style.format({'change': "{:.2%}"})
#result2

In [ ]:
def label (row):
   #'FB','GOOGL','MSFT','JPM','BAC','WFC','JNJ','CVS','MRK','UA','DAL','AAL','MAR','HLT','H' 
    if row['symbol'] == '^DJI' :
        return 'Dow Jones'
    if row['symbol'] == '^GSPC' :
        return 'S&P 500'
    if row['symbol'] == '^IXIC':
        return 'Nasdaq'
    if row['symbol'] == 'FB' :
        return 'Facebook-Tech'
    if row['symbol'] == 'MSFT' :
        return 'Microsoft-Tech'
    if row['symbol'] == 'GOOGL':
        return 'Google-Tech'
    if row['symbol'] == 'JPM' :
        return 'JP Morgan-Financial'
    if row['symbol'] == 'BAC' :
        return 'Bank of America-Financial'
    if row['symbol'] == 'WFC':
        return 'Wells Fargo-Financial'
    if row['symbol'] == 'CVS' :
        return 'CVS-Health'
    if row['symbol'] == 'JNJ' :
        return 'Johnson & Johnson-Health'
    if row['symbol'] == 'MRK':
        return 'Merk-Health'
    if row['symbol'] == 'UAL' :
        return 'United Airline-Airline'
    if row['symbol'] == 'AAL' :
        return 'American Airelin-Airline'
    if row['symbol'] == 'DAL':
        return 'Delta Airline-Airline'
    if row['symbol'] == 'MAR':
        return 'Marriott-Hotel'
    if row['symbol'] == 'HTL' :
        return 'Hilto-Hotel'
    if row['symbol'] == 'H' :
        return 'Hayyt-Hotel'
    return 'Other'


In [ ]:
result2['label'] = result2.apply (lambda row: label(row), axis=1)

In [ ]:
df_index = result2.loc[result['symbol'].isin(['^DJI','^GSPC','^IXIC'])]

#df_index.style.format({'change': "{:.2%}"})

# **Data that specifies ICU Beds by State and County. It also factors in the Seniors (60+) in the County**

In [ ]:
demoGraphicAndICUByCounty = pd.read_csv(file)
#print("Summary of ICU Beds per State and County with Residents (60+) per ICU Bed")
demoGraphicAndICUByCounty = demoGraphicAndICUByCounty.rename(columns = {"ICU Beds":"ICU_Beds","Total Population":"Total_Population","Population Aged 60+":"Population_Aged_60+","Percent of Population Aged 60+":"Percent_Population_Aged_60","Residents Aged 60+ Per Each ICU Bed":"Residents_Per_ICU_Bed"})
#demoGraphicAndICUByCounty

# **Data that specifies US County wise daily COVID19 count. This is a time series data account from 01 / Jan /2020)**

In [ ]:
jhuDailyDataSet=pd.read_csv('./us-counties/us-counties.csv')
#print("Summary of JHU Daily Dataset across the US (Starting Jan 01 2020)")
#jhuDailyDataSet

In [ ]:
countWithNoICUBeds = demoGraphicAndICUByCounty.query("ICU_Beds==0")
#countWithNoICUBeds
countiesWithICUBeds = demoGraphicAndICUByCounty.query("ICU_Beds>0")

## BRIEF TEXT SUMMARY

In [ ]:
comp = pd.read_csv('data/comp.csv', parse_dates=['Start', 'End'])

## Into The World of Pandemics... A Comparison

In this section, we will compare and contrast the five major pandemics in the 21st century.  These pandemics are:

* Covid-19 of 2019
* Ebola of 2014
* MERS of 2012
* H1N1 of 2009
* SARS of 2003

### Overview

The interactive chart below shows the recent five major pandemics in the 21st century along with the seaonsal flu.  Place your cursor over any colored bar to learn more about each pandemic.

In [ ]:
pandemicChart = alt.Chart(comp).mark_bar(size=10.0).encode(
    x = 'Start:T',
    x2 = 'End:T',
    y = 'Epidemic:N',
    color = 'Epidemic',
    tooltip = ['Epidemic', 'Key Symptoms', 'First Detection', 'Global Cases', 'Global Deaths',
               'Transmission', 'Most Affected Groups', 'Treatments Available', 'Vaccines Available',
               'End of Epidemic']
)

In [ ]:
pandemicChart.interactive()

## COVID-19 DATALOAD

In [ ]:
# COVID-19
# --------

# covid_19 dataset
covid_19 = pd.read_csv('data/covid_19_clean_complete.csv', parse_dates=['Date'])

# selecting important columns only
covid_19 = covid_19[['Date', 'Country/Region', 'Confirmed', 'Deaths', 'Recovered']]

# replacing Mainland china with just China
covid_19['Country/Region'] = covid_19['Country/Region'].replace('Mainland China', 'China')

# renaming columns
covid_19.columns = ['Date', 'Country', 'Cases', 'Deaths', 'Recovered']

# group by date and country
covid_19 = covid_19.groupby(['Date', 'Country'])['Cases', 'Deaths', 'Recovered']
covid_19 = covid_19.sum().reset_index()

# latest
c_lat = covid_19[covid_19['Date'] == max(covid_19['Date'])].reset_index()

# latest grouped by country
c_lat_grp = c_lat.groupby('Country')['Cases', 'Deaths', 'Recovered'].sum().reset_index()
c_lat_grp['sqrtCases'] = np.sqrt(c_lat_grp['Cases'])

# nth day
covid_19['nth_day'] = (covid_19['Date'] - min(covid_19['Date'])).dt.days

# day by day
c_dbd = covid_19.groupby('Date')['Cases', 'Deaths', 'Recovered'].sum().reset_index()

# nth day
c_dbd['nth_day'] = covid_19.groupby('Date')['nth_day'].max().values

# mortality rate
c_dbd['Mortality'] = c_dbd['Deaths'] / c_dbd['Cases'] * 100

# no. of countries
temp = covid_19[covid_19['Cases']>0]
c_dbd['n_countries'] = temp.groupby('Date')['Country'].apply(len).values

## EBOLA DATALOAD

In [ ]:
# EBOLA
# ------

# ebola dataset
ebola_14 = pd.read_csv("data/ebola_2014_2016_clean.csv", parse_dates=['Date'])

# selecting important columns only
ebola_14 = ebola_14[['Date', 'Country', 'No. of confirmed, probable and suspected cases',
                     'No. of confirmed, probable and suspected deaths']]

# renaming columns
ebola_14.columns = ['Date', 'Country', 'Cases', 'Deaths']

In [ ]:
# temp = ebola_14.copy()

# for c in ebola_14['Country'].unique():
#     temp_data = ebola_14[ebola_14['Country'] == c]
#     idx = ebola_14.index[ebola_14['Country'] == c]

#     for i in range(len(idx)-1):        
#         if ((temp_data.loc[idx[i+1]]['Date'] - temp_data.loc[idx[i]]['Date']) >= timedelta(days=2)):
#             tracker = temp_data.loc[idx[i]]['Date'] + timedelta(days=1)  
            
#             while tracker < temp_data.loc[idx[i+1]]['Date']:
#                 cases = temp_data.loc[idx[i]]['Cases']
#                 deaths = temp_data.loc[idx[i]]['Deaths']
#                 to_add = pd.DataFrame([{'Date': tracker, 
#                                        'Country': c, 
#                                        'Cases': cases, 
#                                        'Deaths': deaths}])
#                 temp = temp.append(to_add, ignore_index = True)
#                 tracker += timedelta(days=1)

# ebola_14 = temp

In [ ]:
# group by date and country
ebola_14 = ebola_14.groupby(['Date', 'Country'])['Cases', 'Deaths']
ebola_14 = ebola_14.sum().reset_index()

# filling missing values
ebola_14['Cases'] = ebola_14['Cases'].fillna(0)
ebola_14['Deaths'] = ebola_14['Deaths'].fillna(0)

# converting datatypes
ebola_14['Cases'] = ebola_14['Cases'].astype('int')
ebola_14['Deaths'] = ebola_14['Deaths'].astype('int')

# latest
e_lat = ebola_14[ebola_14['Date'] == max(ebola_14['Date'])].reset_index()

# latest grouped by country
e_lat_grp = e_lat.groupby('Country')['Cases', 'Deaths'].sum().reset_index()

# nth day
ebola_14['nth_day'] = (ebola_14['Date'] - min(ebola_14['Date'])).dt.days

# day by day
e_dbd = ebola_14.groupby('Date')['Cases', 'Deaths'].sum().reset_index()

# nth day
e_dbd['nth_day'] = ebola_14.groupby('Date')['nth_day'].max().values

# mortality rate
e_dbd['Mortality'] = e_dbd['Deaths'] / e_dbd['Cases'] * 100

# no. of countries
temp = ebola_14[ebola_14['Cases']>0]
e_dbd['n_countries'] = temp.groupby('Date')['Country'].apply(len).values

ebola_14['Recovered'] = 0
ebola_14 = ebola_14[['Date','Country','Cases','Deaths','Recovered','nth_day']]

## SARS DATALOAD

In [ ]:
# SARS
# ----

# sars dataset
sars_03 = pd.read_csv("data/sars_2003_complete_dataset_clean.csv", parse_dates=['Date'])

# selecting important columns only
sars_03 = sars_03[['Date', 'Country', 'Cumulative number of case(s)', 
                   'Number of deaths', 'Number recovered']]

# renaming columns
sars_03.columns = ['Date', 'Country', 'Cases', 'Deaths', 'Recovered']

# group by date and country
sars_03 = sars_03.groupby(['Date', 'Country'])['Cases', 'Deaths', 'Recovered']
sars_03 = sars_03.sum().reset_index()

In [ ]:
temp = sars_03.copy()

for c in sars_03['Country'].unique():
    temp_data = sars_03[sars_03['Country'] == c]
    idx = sars_03.index[sars_03['Country'] == c]

    for i in range(len(idx)-1):        
        if ((temp_data.loc[idx[i+1]]['Date'] - temp_data.loc[idx[i]]['Date']) >= timedelta(days=2)):
            tracker = temp_data.loc[idx[i]]['Date'] + timedelta(days=1)  
            
            while tracker < temp_data.loc[idx[i+1]]['Date']:
                cases = temp_data.loc[idx[i]]['Cases']
                deaths = temp_data.loc[idx[i]]['Deaths']
                recovered = temp_data.loc[idx[i]]['Recovered']
                to_add = pd.DataFrame([{'Date': tracker, 
                                       'Country': c, 
                                       'Cases': cases, 
                                       'Deaths': deaths,
                                       'Recovered': recovered}])
                temp = temp.append(to_add, ignore_index = True)
                tracker += timedelta(days=1)

sars_03 = temp

In [ ]:
# latest
s_lat = sars_03[sars_03['Date'] == max(sars_03['Date'])].reset_index()

# latest grouped by country
s_lat_grp = s_lat.groupby('Country')['Cases', 'Deaths', 'Recovered'].sum().reset_index()

# nth day
sars_03['nth_day'] = (sars_03['Date'] - min(sars_03['Date'])).dt.days

# day by day
s_dbd = sars_03.groupby('Date')['Cases', 'Deaths', 'Recovered'].sum().reset_index()

# nth day
s_dbd['nth_day'] = sars_03.groupby('Date')['nth_day'].max().values

# mortality rate
s_dbd['Mortality'] = s_dbd['Deaths'] / s_dbd['Cases'] * 100

# no. of countries
temp = sars_03[sars_03['Cases']>0]
s_dbd['n_countries'] = temp.groupby('Date')['Country'].apply(len).values

## H1N1 DATALOAD

In [ ]:
# H1N1
# ----

# h1n1 dataset
h1n1_09 = pd.read_csv("data/Pandemic (H1N1) 2009.csv", parse_dates=['Update Time'])

# retain the date portion of 'Update Time'
h1n1_09['Date'] = h1n1_09['Update Time'].dt.date

# keep relevant columns
h1n1_09 = h1n1_09[['Date', 'Country', 'Cases', 'Deaths']]

In [ ]:
temp = h1n1_09.copy()

for c in h1n1_09['Country'].unique():
    temp_data = h1n1_09[h1n1_09['Country'] == c]
    idx = h1n1_09.index[h1n1_09['Country'] == c]

    for i in range(len(idx)-1):        
        if ((temp_data.loc[idx[i+1]]['Date'] - temp_data.loc[idx[i]]['Date']) >= timedelta(days=2)):
            tracker = temp_data.loc[idx[i]]['Date'] + timedelta(days=1)  
            
            while tracker < temp_data.loc[idx[i+1]]['Date']:
                cases = temp_data.loc[idx[i]]['Cases']
                deaths = temp_data.loc[idx[i]]['Deaths']
                to_add = pd.DataFrame([{'Date': tracker, 
                                       'Country': c, 
                                       'Cases': cases, 
                                       'Deaths': deaths}])
                temp = temp.append(to_add, ignore_index = True)
                tracker += timedelta(days=1)

h1n1_09 = temp

In [ ]:
# latest
h_lat = h1n1_09[h1n1_09['Date'] == max(h1n1_09['Date'])].reset_index()

# latest grouped by country
h_lat_grp = h_lat.groupby('Country')['Cases', 'Deaths'].sum().reset_index()

# nth day calculation and reverse the row order
h1n1_09['nth_day'] = (h1n1_09['Date'] - min(h1n1_09['Date'])).dt.days
h1n1_09 = h1n1_09.iloc[::-1].reset_index(drop=True)

# fill NaN with 0
h1n1_09 = h1n1_09.fillna(0)

# change 'Deaths' column to integers
h1n1_09['Deaths'] = h1n1_09['Deaths'].astype(int)

# day by day
h_dbd = h1n1_09.groupby('Date')['Cases', 'Deaths'].sum().reset_index()

# nth day
h_dbd['nth_day'] = h1n1_09.groupby('Date')['nth_day'].max().values

# mortality rate
h_dbd['Mortality'] = h_dbd['Deaths'] / h_dbd['Cases'] * 100

# no. of countries
temp = h1n1_09[h1n1_09['Cases']>0]
h_dbd['n_countries'] = temp.groupby('Date')['Country'].apply(len).values

## MERS DATALOAD

In [ ]:
# MERS
# ------

# mers dataset
mers_12 = pd.read_csv("data/mers_weekly_clean.csv")
mers_cntry = pd.read_csv("data/mers_country_count_latest.csv")

# transforming data to obtain 'Date'
mers_12['Date'] = mers_12.apply(lambda row: str(row['Year']) + '-W' + str(row['Week']) +'-1', axis=1)
test = datetime.datetime.strptime('2012-W12-1', '%Y-W%W-%w')
mers_12['Date'] = mers_12.apply(lambda row: datetime.datetime.strptime(row['Date'], '%Y-W%W-%w'), axis=1)

# keeping important columns only
mers_12 = mers_12[['Region', 'New Cases', 'Date']]

# renaming and rearranging columns
mers_12.columns = ['Country', 'Cases', 'Date']
mers_12 = mers_12[['Date', 'Country', 'Cases']]

# calculate cumulative cases
temp = pd.DataFrame({'Date':[], 'Country':[], 'Cases':[]})

for c in mers_12.Country.unique():
    temp_df = mers_12[mers_12['Country']==c]
    temp_df['Cases'] = temp_df['Cases'].cumsum()
    temp = temp.append(temp_df, ignore_index = True)
    
mers_12 = temp

In [ ]:
# latest
m_lat = mers_12[mers_12['Date'] == max(mers_12['Date'])].reset_index()

# latest grouped by country
m_lat_grp = m_lat.groupby('Country')['Cases'].sum().reset_index()

# nth day
mers_12['nth_day'] = (mers_12['Date'] - min(mers_12['Date'])).dt.days

# day by day
m_dbd = mers_12.groupby('Date')['Cases'].sum().reset_index()

# nth day
m_dbd['nth_day'] = mers_12.groupby('Date')['nth_day'].max().values

# # no. of countries
# temp = sars_03[sars_03['Cases']>0]
# s_dbd['n_countries'] = temp.groupby('Date')['Country'].apply(len).values

## MERGING DATA

In [ ]:
e_dbd['new_cases'] = e_dbd['Cases'].diff()
c_dbd['new_cases'] = c_dbd['Cases'].diff()
s_dbd['new_cases'] = s_dbd['Cases'].diff()
h_dbd['new_cases'] = h_dbd['Cases'].diff()
m_dbd['new_cases'] = m_dbd['Cases'].diff()

e_dbd['new_deaths'] = e_dbd['Deaths'].diff()
c_dbd['new_deaths'] = c_dbd['Deaths'].diff()
s_dbd['new_deaths'] = s_dbd['Deaths'].diff()
h_dbd['new_deaths'] = h_dbd['Deaths'].diff()

e_dbd['epidemic'] = 'EBOLA'
c_dbd['epidemic'] = 'COVID-19'
s_dbd['epidemic'] = 'SARS'
h_dbd['epidemic'] = 'H1N1'
m_dbd['epidemic'] = 'MERS'

In [ ]:
c_lat_grp["log_Cases"] = np.log(c_lat_grp["Cases"])
e_lat_grp["log_Cases"] = np.log(e_lat_grp["Cases"])
s_lat_grp["log_Cases"] = np.log(s_lat_grp["Cases"])
h_lat_grp["log_Cases"] = np.log(h_lat_grp["Cases"])

c_lat_grp["log_Deaths"] = np.log(c_lat_grp["Deaths"]+1)
e_lat_grp["log_Deaths"] = np.log(e_lat_grp["Deaths"]+1)
s_lat_grp["log_Deaths"] = np.log(s_lat_grp["Deaths"]+1)
h_lat_grp["log_Deaths"] = np.log(h_lat_grp["Deaths"]+1)

In [ ]:
covid_19['Epidemic'] = 'COVID-19'
ebola_14['Epidemic'] = 'EBOLA'
sars_03['Epidemic'] = 'SARS'
h1n1_09['Epidemic'] = 'H1N1'
mers_12['Epidemic'] = 'MERS'

In [ ]:
data = pd.concat([c_dbd, s_dbd, e_dbd, h_dbd, m_dbd])
data.head()

In [ ]:
alt.data_transformers.enable('json')

### Spread Across the Globe

Scroll to the world map of the pandemic of interest.  Place mouse over the country to learn the number of reported cases.  With the mouse over the world map, scroll up and down to zoom in and out.

In [ ]:
covidFig = px.choropleth(c_lat_grp, locations="Country", locationmode='country names',
                    color="Cases", hover_name="Country", 
                    color_continuous_scale="reds", title='COVID-19 REPORTED CASES')
# fig.update(layout_coloraxis_showscale=False)


#-----------------------------------------------------------------------------------------

ebolaFig = px.choropleth(e_lat_grp, locations="Country", locationmode='country names',
                    color="Cases", hover_name="Country", 
                    color_continuous_scale="reds", title='EBOLA 2014 REPORTED CASES')
# fig.update(layout_coloraxis_showscale=False)


#-----------------------------------------------------------------------------------------

sarsFig = px.choropleth(s_lat_grp, locations="Country", locationmode='country names',
                    color="Cases", hover_name="Country", 
                    color_continuous_scale="reds", title='SARS 2003 REPORTED CASES')
# fig.update(layout_coloraxis_showscale=False)


#-----------------------------------------------------------------------------------------

h1n1Fig = px.choropleth(h_lat_grp, locations="Country", locationmode='country names',
                    color="Cases", hover_name="Country", 
                    color_continuous_scale="reds", title='H1N1 2009 REPORTED CASES')
# fig.update(layout_coloraxis_showscale=False)


#-----------------------------------------------------------------------------------------

mersFig = px.choropleth(m_lat_grp, locations="Country", locationmode='country names',
                    color="Cases", hover_name="Country", 
                    color_continuous_scale="reds", title='MERS 2012 REPORTED CASES')
# fig.update(layout_coloraxis_showscale=False)

In [ ]:
covidFig.show()

In [ ]:
ebolaFig.show()

In [ ]:
sarsFig.show()

In [ ]:
h1n1Fig.show()

In [ ]:
mersFig.show()

### Reported Cases By Day

The line chart on the next slide shows the number of reported cases of each pandemic by day.  The view window (narrow window on the bottom) is interactive.  Double click on the view window and drag the mouse to select a timeframe of interest.  Click and hold the mouse and drag left and right to slide the selected timeframe.  With the mouse over the selected timeframe, scroll up and down to widen and shrink the selected timeframe.

In [ ]:
interval_cases = alt.selection_interval(encodings=['x'])

base_cases = alt.Chart(data).mark_line(clip=True).encode(
    x = alt.X('nth_day:Q', scale=alt.Scale(domain=(0,600))),
    y = 'Cases:Q',
    color = 'epidemic'
)

chart_cases = base_cases.properties(
    width=800,
    height=300
).encode(
    x=alt.X('nth_day:Q', scale=alt.Scale(domain=interval_cases.ref()))
)

view_cases = base_cases.properties(
    width=800,
    height=50,
    selection=interval_cases
)

In [ ]:
chart_cases & view_cases

### Reported Deaths By Day

The line chart on the next slide shows the number of reported deaths of each pandemic by day.  The view window (narrow window on the bottom) is interactive.  Double click on the view window and drag the mouse to select a timeframe of interest.  Click and hold the mouse and drag left and right to slide the selected timeframe.  With the mouse over the selected timeframe, scroll up and down to widen and shrink the selected timeframe.

In [ ]:
interval_deaths = alt.selection_interval(encodings=['x'])

base_deaths = alt.Chart(data).mark_line().encode(
    x = alt.X('nth_day:Q', scale=alt.Scale(domain=(0,600))),
    y = 'Deaths:Q',
    color = 'epidemic'
)

chart_deaths = base_deaths.properties(
    width=800,
    height=300
).encode(
    x=alt.X('nth_day:Q', scale=alt.Scale(domain=interval_deaths.ref()))
)

view_deaths = base_deaths.properties(
    width=800,
    height=50,
    selection=interval_deaths
)

In [ ]:
chart_deaths & view_deaths

### Mortality Rate By Day

The line chart on the next slide shows the mortality rate of each pandemic by day.  The view window (narrow window on the bottom) is interactive.  Double click on the view window and drag the mouse to select a timeframe of interest.  Click and hold the mouse and drag left and right to slide the selected timeframe.  With the mouse over the selected timeframe, scroll up and down to widen and shrink the selected timeframe.

In [ ]:
interval_deaths = alt.selection_interval(encodings=['x'])

base_deaths = alt.Chart(data).mark_line().encode(
    x = alt.X('nth_day:Q', scale=alt.Scale(domain=(0,600))),
    y = 'Mortality:Q',
    color = 'epidemic'
)

chart_deaths = base_deaths.properties(
    width=800,
    height=300
).encode(
    x=alt.X('nth_day:Q', scale=alt.Scale(domain=interval_deaths.ref()))
)

view_deaths = base_deaths.properties(
    width=800,
    height=50,
    selection=interval_deaths
)

In [ ]:
chart_deaths & view_deaths

### Reported Cases By Country

Scroll to the tree map of the pandemic of interest.  Place mouse over the country to learn the number of reported cases.

In [ ]:
import plotly.io as pio

covidFig = px.treemap(c_lat_grp.sort_values(by='Cases', ascending=False).reset_index(drop=True), 
                 path=["Country"], values="Cases", title='COVID-19 Cases',
                 color_discrete_sequence = px.colors.qualitative.Set1)


# fig = px.treemap(c_lat_grp.sort_values(by='Cases', ascending=False).reset_index(drop=True), 
#                  path=["Country"], values="sqrtCases", title='COVID-19',
#                  color_discrete_sequence = px.colors.qualitative.Set1)
# fig.show()

# pio.write_html(covidFig, file='index_covid_19_cases.html', auto_open=False)

ebolaFig = px.treemap(e_lat_grp.sort_values(by='Cases', ascending=False).reset_index(drop=True), 
                 path=["Country"], values="Cases", title='EBOLA Cases',
                 color_discrete_sequence = px.colors.qualitative.Set1)


# pio.write_html(ebolaFig, file='index_ebola_14_cases.html', auto_open=False)

sarsFig = px.treemap(s_lat_grp.sort_values(by='Cases', ascending=False).reset_index(drop=True), 
                 path=["Country"], values="Cases", title='SARS Cases',
                 color_discrete_sequence = px.colors.qualitative.Set1)


# pio.write_html(sarsFig, file='index_sars_03_cases.html', auto_open=False)

h1n1Fig = px.treemap(h_lat_grp.sort_values(by='Cases', ascending=False).reset_index(drop=True), 
                 path=["Country"], values="Cases", title='H1N1 Cases',
                 color_discrete_sequence = px.colors.qualitative.Set1)


# pio.write_html(h1n1Fig, file='index_h1n1_09_cases.html', auto_open=False)

mersFig = px.treemap(mers_cntry, 
                 path=["Country"], values="Confirmed", title='MERS Cases',
                 color_discrete_sequence = px.colors.qualitative.Set1)

# pio.write_html(mersFig, file='index_mers_12_cases.html', auto_open=False)

In [ ]:
covidFig.show()

In [ ]:
ebolaFig.show()

In [ ]:
sarsFig.show()

In [ ]:
h1n1Fig.show()

In [ ]:
mersFig.show()

### Reported Deaths By Country

Scroll to the tree map of the pandemic of interest.  Place mouse over the country to learn the number of reported deaths.

In [ ]:
covidFig = px.treemap(c_lat_grp.sort_values(by='Deaths', ascending=False).reset_index(drop=True), 
                 path=["Country"], values="Deaths", title='COVID-19 Deaths',
                 color_discrete_sequence = px.colors.qualitative.Set1)


# fig = px.treemap(c_lat_grp.sort_values(by='Cases', ascending=False).reset_index(drop=True), 
#                  path=["Country"], values="sqrtCases", title='COVID-19',
#                  color_discrete_sequence = px.colors.qualitative.Set1)
# fig.show()

# pio.write_html(covidFig, file='index_covid_19_deaths.html', auto_open=False)

ebolaFig = px.treemap(e_lat_grp.sort_values(by='Deaths', ascending=False).reset_index(drop=True), 
                 path=["Country"], values="Deaths", title='EBOLA Deaths',
                 color_discrete_sequence = px.colors.qualitative.Set1)


# pio.write_html(ebolaFig, file='index_ebola_14_deaths.html', auto_open=False)

sarsFig = px.treemap(s_lat_grp.sort_values(by='Deaths', ascending=False).reset_index(drop=True), 
                 path=["Country"], values="Deaths", title='SARS Deaths',
                 color_discrete_sequence = px.colors.qualitative.Set1)


# pio.write_html(sarsFig, file='index_sars_03_deaths.html', auto_open=False)

h1n1Fig = px.treemap(h_lat_grp.sort_values(by='Deaths', ascending=False).reset_index(drop=True), 
                 path=["Country"], values="Deaths", title='H1N1 Deaths',
                 color_discrete_sequence = px.colors.qualitative.Set1)


# pio.write_html(h1n1Fig, file='index_h1n1_09_deaths.html', auto_open=False)

mersFig = px.treemap(mers_cntry, 
                 path=["Country"], values="Confirmed", title='MERS',
                 color_discrete_sequence = px.colors.qualitative.Set1)

# pio.write_html(mersFig, file='index_mers_12.html', auto_open=True)

In [ ]:
covidFig.show()

In [ ]:
ebolaFig.show()

In [ ]:
sarsFig.show()

In [ ]:
h1n1Fig.show()

In [ ]:
mersFig.show()

### COVID-19 Vulnerable Counties in USA - In this interactive chart series we are going to see how ICU beds are distributed across Counties. Goal is to show the most vulnerable counties having least to no ICU beds for the population it is serving. 

## Most Vulnerable Counties having no ICU Beds in US

In [ ]:
#print("Most Vulnerable Counties having No ICU Beds")
chart = alt.Chart(countWithNoICUBeds).mark_circle(size=50).encode(
    x='Total_Population',
    y='Population_Aged_60+',
    color='State',
    tooltip=['State', 'County','Percent_Population_Aged_60','Population_Aged_60+','Total_Population']
).properties(
    title='Vulnerable Counties in US having Zero ICU Bed per County'
)
chart.configure_title(
    fontSize=20,
    font='Arial',
    anchor='start',
    color='gray'
)

## Vulnerable Counties in US having atleast one ICU Bed, ratio of Seniors to population along with bed to individuals ratio

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:

jhuDailyDataSet = jhuDailyDataSet.rename(columns={'state':'State','county':'County'})
jhuAggregateDataSet = jhuDailyDataSet.groupby(['State','County'])
jhuAggregateDataSetAggregate = jhuAggregateDataSet.agg(np.sum)
#jhuAggregateDataSetAggregate
mergedDataSet = pd.merge(jhuAggregateDataSetAggregate,countiesWithICUBeds,left_on='State',right_on='State')

chart2 = alt.Chart(mergedDataSet).mark_circle(size=50).encode(
    x='cases',
    y='Residents_Per_ICU_Bed',
    color='State',
    tooltip=['State', 'County','Percent_Population_Aged_60','Population_Aged_60+','Total_Population','cases','deaths','Residents_Per_ICU_Bed']
).properties(
    title='Vulnerable Counties in US having atleast One ICU Bed per County'
)

chart2.configure_title(
    fontSize=20,
    font='Arial',
    anchor='start',
    color='gray'
)

## Price changes in different sectors and S&P 500 index
### Price of leading company in industries impacted are listed, price of United Airline and Marriott dropped significantly while Microsoft and Johnson & Johnson are relatively stable

In [ ]:
df = result2.loc[result2['symbol'].isin(['^GSPC','MSFT', 'JPM', 'JNJ', 'UAL','MAR'])]



# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['date'], empty='none')


# The basic line
line = alt.Chart(df).mark_line(interpolate='basis').encode(
    x= alt.X('date:T', title='Sectors and Index'),
    y='change:Q',
    color='label:N'
    )

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(df).mark_point().encode(
    x='date:T',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'change:Q', alt.value(' '))#alt.X('PercentOfTotal:Q', axis=alt.Axis(format='.0%'))
)
#alt.X('PercentOfTotal:Q', axis=alt.Axis(format='.0%'))
# Draw a rule at the location of the selection
rules = alt.Chart(df).mark_rule(color='gray').encode(
    x='date:T',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=600, height=300
)
#chart.save(html)

## Changes of indexes
### Dow Jones dropped significantly after confirmed case number increased to more than 1000. Market became opstimastic from then end of March

In [ ]:
source2 = [{
            "start": "2020-01-04",
            "end": "2020-01-22",
            "event": "No Confirmed case in US"
          },
          {
            "start": "2020-01-23",
            "end": "2020-03-10",
            "event": "Confimed case <1000 in US"
          },
          {
           "start": "2020-03-11",
            "end": "2020-04-08",
            "event": "Confimed case >1000 in US"   
          }  ]


source2 = alt.pd.DataFrame(source2)

highlight = alt.selection(type='single', on='mouseover',
                          fields=['label'], nearest=True)

base = alt.Chart(df_index).encode(
    x= alt.X('date:T', title='Indexes'),
    y='change:Q',
    #y=alt.Y('change:Q', axis=alt.Axis(format='.0%'))
    color=alt.Color('label:N', scale=alt.Scale(scheme='tableau20'))
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=600
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3))
)
rect = alt.Chart(source2).mark_rect().encode(
    x='start:T',
    x2='end:T',
    color=alt.Color('event:N', legend=alt.Legend(title="Legend"))
)

rect + points + lines

## Price of Technology stocks droped about 20% in the week of March 15

In [ ]:
#Tech

df_IT = result.loc[result['symbol'].isin(['FB','GOOGL','MSFT'])]


highlight = alt.selection(type='single', on='mouseover',
                          fields=['symbol'], nearest=True)

base = alt.Chart(df_IT).encode(
    x= alt.X('date:T', title='Technology'),
    y='change:Q',
    color=alt.Color('symbol:N', scale=alt.Scale(scheme='tableau10'))
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=600
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3))
)
rect = alt.Chart(source2).mark_rect().encode(
    x='start:T',
    x2='end:T',
    color=alt.Color('event:N', legend=alt.Legend(title="Legend"))
)

rect + points + lines

## Price of Financial stocks droped about 40% in the week of March 15

In [ ]:
#Financial

df_IT = result.loc[result['symbol'].isin(['JPM','BAC','WFC'])]


highlight = alt.selection(type='single', on='mouseover',
                          fields=['symbol'], nearest=True)

base = alt.Chart(df_IT).encode(
    x= alt.X('date:T', title='Financial'),
    y='change:Q',
    color=alt.Color('symbol:N', scale=alt.Scale(scheme='tableau10'))
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=600
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3))
)
rect = alt.Chart(source2).mark_rect().encode(
    x='start:T',
    x2='end:T',
    color=alt.Color('event:N', legend=alt.Legend(title="Legend"))
)

rect + points + lines

## Price of Healthcare stocks droped about 20% in the week of March 15

In [ ]:
#Health

df_IT = result.loc[result['symbol'].isin(['JNJ','MRK','CVS'])]


highlight = alt.selection(type='single', on='mouseover',
                          fields=['symbol'], nearest=True)

base = alt.Chart(df_IT).encode(
    x= alt.X('date:T', title='Healthcare'),
    y='change:Q',
    color=alt.Color('symbol:N', scale=alt.Scale(scheme='tableau10'))
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=600
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3))
)
rect = alt.Chart(source2).mark_rect().encode(
    x='start:T',
    x2='end:T',
    color=alt.Color('event:N', legend=alt.Legend(title="Legend"))
)

rect + points + lines

## Price of Airlines stocks droped about 60% in the week of March 15


In [ ]:
#Airline

df_IT = result.loc[result['symbol'].isin(['UAL','DAL','AAL'])]


highlight = alt.selection(type='single', on='mouseover',
                          fields=['symbol'], nearest=True)

base = alt.Chart(df_IT).encode(
    x= alt.X('date:T', title='Airlines'),
    y='change:Q',
    color=alt.Color('symbol:N', scale=alt.Scale(scheme='tableau10'))
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=600
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3))
)
rect = alt.Chart(source2).mark_rect().encode(
    x='start:T',
    x2='end:T',
    color=alt.Color('event:N', legend=alt.Legend(title="Legend"))
)

rect + points + lines

## Price of Hotel stocks droped about 50% in the week of March 15

In [ ]:
#Hotel

df_IT = result.loc[result['symbol'].isin(['MAR','HLT','H'])]


highlight = alt.selection(type='single', on='mouseover',
                          fields=['symbol'], nearest=True)

base = alt.Chart(df_IT).encode(
    x= alt.X('date:T', title='Hotels'),
    y='change:Q',
    color=alt.Color('symbol:N', scale=alt.Scale(scheme='tableau10'))
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=600
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3))
)
rect = alt.Chart(source2).mark_rect().encode(
    x='start:T',
    x2='end:T',
    color=alt.Color('event:N', legend=alt.Legend(title="Legend"))
)

rect + points + lines